In [25]:
import pandas as pd
import numpy as np
import os
import plotly.express as px
from tqdm import tqdm
import json
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier
pd.options.plotting.backend = "plotly"

https://disk.yandex.ru/d/b5NdFMfnDEVyTQ

In [2]:
df = pd.read_csv('sub_final_6.csv', index_col='Unnamed: 0')
df = df.drop('trans_day', axis=1)

In [3]:
df

,amount,target,mcc_742,mcc_1711,mcc_1731,mcc_1799,mcc_2741,mcc_3000,mcc_3351,mcc_3501,...,city_1,city_2,city_3,city_4,city_5,city_6,city_7,city_8,city_9,city_10
d1bbbc9a0e0410d3cf12a3d2f44f3450,"[-1808.56, -3390.41, -144.5, -3542.3, -3542.70...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
81c7349f7bdcb661bc7d5f5f0fdac6aa,"[7954.51, -216.74, -252.19000000000003, -362.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
fc0c7c937f139bd6d1c5b5d0a0023dd3,"[-12293.56, 10484.57, 21693.390000000003, 1084...",NaN,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3495487d5beaa5caaa0682fddc38379a,"[49.52, -1445.24, 506.98, -6.0, 6901.08, 181.0...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
07e279042b1da23710fa2f65034d30b9,"[45.29, -691.78, 449.09, 329.13, -2182.44, 325...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
a517cdee04c890b311fc876a325dba99,"[-36.61, 35.18]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
b6714c0ca16193cbd419623eb70b1488,[-73.31],1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
ee4c5d78cbb27771979714ffa903ef5d,[72.71000000000001],0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
d428ad799c5790b036beb67cc1b81075,"[-14463.42, -18078.07]",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [4]:
amount_len = []
for el in list(df['amount']):
    amount_len.append(len(el))

In [5]:
mean_p = []
median_p = []
mean_n = []
median_n = []
std_p = []
std_n = []
min_n = []
min_p = []
max_n = []
max_p = []
sum_p = []
sum_n = []
stds = []
for el in list(df['amount']):
    el = [float(x) for x in el.replace('[', '').replace(']', '').split(',')]
    el = np.array(el)
    stds.append(np.std(el))
    el_p = el[el > 0]
    el_n = el[el < 0]
    if len(el_p) == 0:
        el_p = np.array([0])
    if len(el_n) == 0:
        el_n = np.array([0])
    mean_p.append(np.mean(el_p))
    mean_n.append(np.mean(el_n))
    median_p.append(np.median(el_p))
    median_n.append(np.median(el_n))
    std_p.append(np.std(el_p))
    std_n.append(np.std(el_n))
    min_n.append(np.min(el_n))
    min_p.append(np.min(el_p))
    max_n.append(np.max(el_n))
    max_p.append(np.max(el_p))
    sum_p.append(np.sum(el_p))
    sum_n.append(np.sum(el_n))

In [6]:
#df['number_of_purch'] = amount_len
df['mean_p'] = mean_p
df['mean_n'] = mean_n
df['median_p'] = median_p
df['median_n'] = median_n
#df['std_p'] = std_p
#df['std_n'] = std_n
df['std'] = stds
df['min_p'] = min_p
df['min_n'] = min_n
df['max_p'] = max_p
df['max_n'] = max_n
#df['sum_p'] = sum_p
#df['sum_n'] = sum_n

In [7]:
df = df.dropna()

In [8]:
X = df.drop(['amount', 'target'], axis=1)
y = df['target']

In [19]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [29]:
model = CatBoostClassifier(iterations=5000, depth=5, learning_rate=0.01, l2_leaf_reg=1)

In [30]:
model.fit(X, y)

0:	learn: 0.6911569	total: 11.2ms	remaining: 56.1s
1:	learn: 0.6884505	total: 19.5ms	remaining: 48.9s
2:	learn: 0.6858035	total: 27.9ms	remaining: 46.4s
3:	learn: 0.6832039	total: 35.9ms	remaining: 44.9s
4:	learn: 0.6810038	total: 44.3ms	remaining: 44.3s
5:	learn: 0.6785753	total: 53.5ms	remaining: 44.5s
6:	learn: 0.6763270	total: 64.6ms	remaining: 46.1s
7:	learn: 0.6741082	total: 74.4ms	remaining: 46.4s
8:	learn: 0.6720893	total: 83.6ms	remaining: 46.4s
9:	learn: 0.6696040	total: 91.5ms	remaining: 45.6s
10:	learn: 0.6675394	total: 99.3ms	remaining: 45s
11:	learn: 0.6656709	total: 107ms	remaining: 44.5s
12:	learn: 0.6639145	total: 115ms	remaining: 44s
13:	learn: 0.6619255	total: 122ms	remaining: 43.5s
14:	learn: 0.6598668	total: 130ms	remaining: 43.2s
15:	learn: 0.6577223	total: 138ms	remaining: 42.9s
16:	learn: 0.6556595	total: 146ms	remaining: 42.7s
17:	learn: 0.6537785	total: 154ms	remaining: 42.5s
18:	learn: 0.6518874	total: 161ms	remaining: 42.3s
19:	learn: 0.6499853	total: 169ms	

In [31]:
roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])

0.9954811720729535

In [32]:
model.save_model('cat_full_dataset',
           format="cbm",
           export_parameters=None,
           pool=None)

In [33]:
model.predict_proba(X_test)[:, 1]

array([0.21722605, 0.21134572, 0.0918437 , ..., 0.13996452, 0.52598022,
       0.33084433])